In [6]:
pip install langchain langchain-community pdfplumber pymupdf faiss-cpu sentence-transformers

  Using cached langchain_community-0.3.19-py3-none-any.whl.metadata (2.4 kB)
  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
  Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
  Using cached langchain_core-0.3.41-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain-0.3.20-py3-none-any.whl.metadata (7.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (48 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached nvidia_cuda_nvrt

In [7]:
!pip install pymupdf



```
# This is formatted as code
```



In [8]:
import fitz

In [9]:
import os
import fitz  # PyMuPDF for PDF parsing
import pdfplumber
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Function to extract text from PDF
def extract_text_from_pdf(Docx_path):
    text = ""
    try:
        # Using PyMuPDF (fitz)
        with fitz.open(Docx_path) as doc:
            for page in doc:
                text += page.get_text("text") + "\n"
    except Exception as e:
        print("Error with PyMuPDF, trying pdfplumber:", e)
        try:
            # Using pdfplumber as an alternative
            with pdfplumber.open(Docx_path) as pdf:
                text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
        except Exception as e:
            print("Error extracting text from PDF:", e)
    return text

In [11]:
# Function to store text in FAISS vector database
def store_in_vector_db(text):
    # Split text into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_text(text)

    # Load embeddings model
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Store text in FAISS vector store
    vector_store = FAISS.from_texts(texts, embedding_model)

    # Save FAISS index for later use
    vector_store.save_local("faiss_index")
    print("Vector database saved!")


In [12]:
# Main function
def main():
    Docx_path = "/content/drive/MyDrive/Hands on/transformers.docx"

    if not os.path.exists(Docx_path):
        print("File not found!")
        return

    print("Extracting text from PDF...")
    text = extract_text_from_pdf(Docx_path)

    if text:
        print("Text extracted successfully!")
        print("Storing in vector database...")
        store_in_vector_db(text)
    else:
        print("Failed to extract text.")

if __name__ == "__main__":
    main()


Extracting text from PDF...
Text extracted successfully!
Storing in vector database...


<ipython-input-11-cb1e4f94175a>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector database saved!


In [13]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load FAISS index with safe deserialization
vector_store = FAISS.load_local(
    "faiss_index",
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    allow_dangerous_deserialization=True  # Add this line
)

# Test query
query = "what is a transformer?"
results = vector_store.similarity_search(query, k=3)
print(results)

[Document(id='69880961-328f-4de9-b275-2f26f26385b7', metadata={}, page_content='Topic\nDefinition\nAdvantages\nDisadvantages\nApplications\nTransformers\nTransformation of data leads to efficiency\n.Scalability\n.Automation\n.Better context understanding in NLP.\nHigh memory usage\nImage classification\nAttention\nModel selectively focus on relevant parts of input sequences.\nParallel processing\n.Training instability\n.Lack of interpretability\n.Text summarization\n.Object detection\nEncoder\nIt process the input layer and convert it into meaningful'), Document(id='d6beb4f0-6038-4a67-94bf-4f29ca30d3dc', metadata={}, page_content='numerical representation.\n.Decoding is the process of reconstructing images,or translating\nlanguages\n.Improved accuracy\n.Handling noisy data or missing data\n.Loss of interpretability data dependency'), Document(id='1bbdb270-dcfa-4cad-a241-8f2d9477317b', metadata={}, page_content='representation while reducing redundancy.\n.Dimensionality reduction\n.Impr